In [345]:
import re, orodja
import pandas as pd
file = 'osu_index.txt'
aux_file = 'data-auxf.txt'
page = 'https://osu.ppy.sh/beatmapsets?sort=difficulty_desc&s=ranked'

V tega bomo dal regex-e

In [346]:
curr_re = r'\{"beatmapsets":\[(.*?)\]\}\]'
data_block_re = r'\{.*?\}\]\}'
data_re = re.compile(
    r'\{'
    r'"artist":(?P<artist>.*?)?,.*?,'
    r'"creator":(?P<creator>.*)?,'
    r'"favourite_count":(?P<fav_count>\d*)?,.*?,'
    r'"id":(?P<id>\d*)?,.*?,'
    r'"play_count":(?P<play_count>\d*)?,.*?,'
    r'"status":(?P<status>.*)?,'
    r'"title":(?P<title>.*?)?,.*?,'
    r'"bpm":(?P<bpm>\d*)?,.*?'
    r'"tags":(?P<tags>.*?)?,'
    r'"beatmaps":\[(?P<beatmapset>.*?)?\]'
    r'\}',
    flags=re.DOTALL
)

beatmapset_re = re.compile(
    r'\{.*'
    r'"difficulty_rating":(?P<difficulty_rating>.*?),.*?,'
    r'"total_length":(?P<map_len>\d*?),.*?,'
    r''
    r'\}',
    flags=re.DOTALL
)

In [347]:
artist_re = r''
creator_re = r''
favourite_count = r''
id_re = r''
play_count_re = r''
status_re = r''
title_re = r''
bpm_re = r''

# Tukej delamo posamezne odseke kode

In [348]:
def scrape(fin: str, out: str) -> None:
    regex = re.compile(curr_re, flags=re.DOTALL) #re.DOTALL postav piko na vse, vkljucno s novo vrstico
    with open(fin) as f:
        stuff = f.read()
        with open(out, 'w', encoding='UTF8')as o:
            print(re.findall(regex, stuff), file=o)

In [349]:
def make_blocks(filename: str) -> list:
    l = []
    with open(filename) as f:
        stuff= f.read()
    regex = re.compile(data_block_re, flags=re.DOTALL)
    for block in re.findall(regex, stuff):
        l.append(block)
    return l

In [357]:
def scrape_block(block: str) -> dict:
    if not block: return {}
    map = data_re.search(block).groupdict()
    map['fav_count'] = int(map.get('fav_count', 0))
    map['id'] = int(map.get('id', 0))
    map['play_count'] = int(map.get('play_count', 0))
    map['bpm'] = int(map['bpm']) 
    map['tags'] = map['tags'].split(' ')
    map['beatmapset'] = map.get('beatmapset', '').split('}')
    return map

In [358]:
def error_check(block: str) -> bool: #eni so prazni iz nekga razloga
    try:
        scrape_block(block)
        return True
    except:
        print('Empty match!')
        return False

Ta del nadomešča (if \_\_name__ == '\_\_main__')

In [360]:
orodja.shrani_spletno_stran(page, file, vsili_prenos=True)
scrape(file, aux_file)
data_blocks = make_blocks(aux_file)
data = []
for i, block in enumerate(data_blocks):
    if not error_check(block):
        print(i)
        continue
    data.append(scrape_block(block))
orodja.zapisi_json(data, 'data.json')
data_format = [
    'id', 
    'creator',
    'artist', 
    'fav_count', 
    'play_count', 
    'status', 
    'title', 
    'bpm', 
    'tags', 
    'beatmapset'
    ]
orodja.zapisi_csv(data, data_format, 'data.csv')

Empty match!
2
Empty match!
29
Empty match!
44
Empty match!
46
Empty match!
47


### Random stuff

In [353]:
data_blocks[26]

'{"artist":"M-Project feat. Mayumi Morinaga","artist_unicode":"M-Project feat. \\\\u68ee\\\\u6c38\\\\u771f\\\\u7531\\\\u7f8e","covers":{"cover":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1817236\\\\/covers\\\\/cover.jpg?1667070299","cover@2x":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1817236\\\\/covers\\\\/cover@2x.jpg?1667070299","card":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1817236\\\\/covers\\\\/card.jpg?1667070299","card@2x":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1817236\\\\/covers\\\\/card@2x.jpg?1667070299","list":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1817236\\\\/covers\\\\/list.jpg?1667070299","list@2x":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1817236\\\\/covers\\\\/list@2x.jpg?1667070299","slimcover":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1817236\\\\/covers\\\\/slimcover.jpg?1667070299","slimcover@2x":"https:\\\\/\\\\/assets.ppy.sh\\\\/beatmaps\\\\/1817236\\\\/covers\\\\/slimcover@2x.jpg?1667070299"},"creator":"Slyme","favo